# 데이터셋 전처리

- 유사한 뉴스 기사들 제거 (문제: 기사 하나 남기고 유사한것만 제거되는게 아닌 전부 제거가 됨..)
- 처벌 컬럼 생성 (..형으로 시작하는 단어나, 무기징역, 그리고 징역일 경우 몇년 까지 포함)
- 정규표현식 후 명사 추출 및 컬럼 생성
- 피의자 나이대 컬럼 생성
- 국내로 제한 (국가 이름들이 나열된 리스트를 웹에서 클롤링해서 국가 이름이 언급될 시 제거)
- 피해자 컬럼 생성 (살인범죄사전)
- 가해자 컬럼 생성 (살인범죄사전)
- 피해자와 가해자의 관계를 파악해서 둘 중 하나가 Nan값인 경우 채워놓기 (예를 들어, 피해자가 부모면 가해자가 자식)
- Nan 값이 없는 행(뉴스기사)들만 추출하여 최종 데이터 프레임 생성

-----------

## 1. 데이터셋 불러오기

In [77]:
import pandas as pd 

pd.set_option('display.max_colwidth', None)

In [78]:
df1 = pd.read_csv('./news_df_1.csv', encoding='utf-8', index_col = 0)
df2 = pd.read_csv('./news_df_2.csv', encoding='utf-8', index_col = 0)
df3 = pd.read_csv('./news_df_3.csv', encoding='utf-8', index_col = 0)
df4 = pd.read_csv('./news_df_4.csv', encoding='utf-8', index_col = 0)
df5 = pd.read_csv('./news_df_5.csv', encoding='utf-8', index_col = 0)
df6 = pd.read_csv('./news_df_6.csv', encoding='utf-8', index_col = 0)
df7 = pd.read_csv('./news_df_7.csv', encoding='utf-8', index_col = 0)
df8 = pd.read_csv('./news_df_8.csv', encoding='utf-8', index_col = 0)
df9 = pd.read_csv('./news_df_9.csv', encoding='utf-8', index_col = 0)
df10 = pd.read_csv('./news_df_10.csv', encoding='utf-8', index_col = 0)
df11 = pd.read_csv('./news_df_11.csv', encoding='utf-8', index_col = 0)
df12 = pd.read_csv('./news_df_12.csv', encoding='utf-8', index_col = 0)

In [79]:
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12])

In [80]:
df

,Unnamed: 0,date,title,url
0,0.0,2022.01.31.,'제2의 자두 사건'...고양이 살해범 잡아주세요,https://www.ytn.co.kr/_ln/0115_202201312029383965
1,1.0,2022.01.31.,"""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수",http://www.newsis.com/view/?id=NISX20220131_0001743534&cID=10812&pID=10800
2,2.0,2022.01.31.,"이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”",http://news.kmib.co.kr/article/view.asp?arcid=0016723451&code=61111111&cp=nv
3,3.0,2022.01.31.,[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대,https://www.news1.kr/articles/?4570950
4,4.0,2022.01.30.,"[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?""",https://www.news1.kr/articles/?4570040
...,...,...,...,...
995,NaN,2021.12.01.,"""동물에게도 못할 범행""…검찰, 생후 20개월 학대 살해범에 사형 구형",https://www.nocutnews.co.kr/news/5666447
996,NaN,2021.12.09.,의처증 남편 살해한 여인의 기구한 운명…'미친.사랑.X',http://www.newsis.com/view/?id=NISX20211209_0001681606&cID=10601&pID=10600
997,NaN,2021.12.15.,[사건·사고24시] ‘세 모녀 살해’ 김태현 오늘 항소심 첫 재판·“왜 내 아내 또 만나” 내연남 때려 숨지게 한 40대 外,https://www.etoday.co.kr/news/view/2087165
998,NaN,2021.12.07.,아동학대 사망 최대 징역 22년 6개월,https://www.ccdailynews.com/news/articleView.html?idxno=2098943


In [81]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [82]:
df.reset_index(drop=True, inplace=True)

------------------

## 2. 유사 뉴스 기사 제거

In [83]:
import difflib

In [84]:
df['index'] = df.index

In [85]:
df['index'] = df['index'].astype(str)+'$'

In [86]:
df['index_similarity'] = df['index']+df['title']

In [87]:
def similarites(row): # change similiar titles to indexs
    similarities_list  = difflib.get_close_matches(row, list(df[df.index_similarity!=row].index_similarity), cutoff = 0.8)
    
    index_lst =[]
    if len(similarities_list) > 1:
        for i in similarities_list:
            similar =  re.findall('^(\d{1,})[$]',i)
            index_lst.append(similar)
            
    return index_lst

In [88]:
df

,date,title,url,index,index_similarity
0,2022.01.31.,'제2의 자두 사건'...고양이 살해범 잡아주세요,https://www.ytn.co.kr/_ln/0115_202201312029383965,0$,0$'제2의 자두 사건'...고양이 살해범 잡아주세요
1,2022.01.31.,"""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수",http://www.newsis.com/view/?id=NISX20220131_0001743534&cID=10812&pID=10800,1$,"1$""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수"
2,2022.01.31.,"이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”",http://news.kmib.co.kr/article/view.asp?arcid=0016723451&code=61111111&cp=nv,2$,"2$이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”"
3,2022.01.31.,[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대,https://www.news1.kr/articles/?4570950,3$,3$[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대
4,2022.01.30.,"[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?""",https://www.news1.kr/articles/?4570040,4$,"4$[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?"""
...,...,...,...,...,...
9076,2021.12.01.,"""동물에게도 못할 범행""…검찰, 생후 20개월 학대 살해범에 사형 구형",https://www.nocutnews.co.kr/news/5666447,9076$,"9076$""동물에게도 못할 범행""…검찰, 생후 20개월 학대 살해범에 사형 구형"
9077,2021.12.09.,의처증 남편 살해한 여인의 기구한 운명…'미친.사랑.X',http://www.newsis.com/view/?id=NISX20211209_0001681606&cID=10601&pID=10600,9077$,9077$의처증 남편 살해한 여인의 기구한 운명…'미친.사랑.X'
9078,2021.12.15.,[사건·사고24시] ‘세 모녀 살해’ 김태현 오늘 항소심 첫 재판·“왜 내 아내 또 만나” 내연남 때려 숨지게 한 40대 外,https://www.etoday.co.kr/news/view/2087165,9078$,9078$[사건·사고24시] ‘세 모녀 살해’ 김태현 오늘 항소심 첫 재판·“왜 내 아내 또 만나” 내연남 때려 숨지게 한 40대 外
9079,2021.12.07.,아동학대 사망 최대 징역 22년 6개월,https://www.ccdailynews.com/news/articleView.html?idxno=2098943,9079$,9079$아동학대 사망 최대 징역 22년 6개월


In [89]:
df['title_similar_index'] = df.index_similarity.apply(lambda row : similarites(row))

In [90]:
df

,date,title,url,index,index_similarity,title_similar_index
0,2022.01.31.,'제2의 자두 사건'...고양이 살해범 잡아주세요,https://www.ytn.co.kr/_ln/0115_202201312029383965,0$,0$'제2의 자두 사건'...고양이 살해범 잡아주세요,[]
1,2022.01.31.,"""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수",http://www.newsis.com/view/?id=NISX20220131_0001743534&cID=10812&pID=10800,1$,"1$""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수",[]
2,2022.01.31.,"이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”",http://news.kmib.co.kr/article/view.asp?arcid=0016723451&code=61111111&cp=nv,2$,"2$이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”",[]
3,2022.01.31.,[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대,https://www.news1.kr/articles/?4570950,3$,3$[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대,[]
4,2022.01.30.,"[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?""",https://www.news1.kr/articles/?4570040,4$,"4$[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?""",[]
...,...,...,...,...,...,...
9076,2021.12.01.,"""동물에게도 못할 범행""…검찰, 생후 20개월 학대 살해범에 사형 구형",https://www.nocutnews.co.kr/news/5666447,9076$,"9076$""동물에게도 못할 범행""…검찰, 생후 20개월 학대 살해범에 사형 구형",[]
9077,2021.12.09.,의처증 남편 살해한 여인의 기구한 운명…'미친.사랑.X',http://www.newsis.com/view/?id=NISX20211209_0001681606&cID=10601&pID=10600,9077$,9077$의처증 남편 살해한 여인의 기구한 운명…'미친.사랑.X',[]
9078,2021.12.15.,[사건·사고24시] ‘세 모녀 살해’ 김태현 오늘 항소심 첫 재판·“왜 내 아내 또 만나” 내연남 때려 숨지게 한 40대 外,https://www.etoday.co.kr/news/view/2087165,9078$,9078$[사건·사고24시] ‘세 모녀 살해’ 김태현 오늘 항소심 첫 재판·“왜 내 아내 또 만나” 내연남 때려 숨지게 한 40대 外,[]
9079,2021.12.07.,아동학대 사망 최대 징역 22년 6개월,https://www.ccdailynews.com/news/articleView.html?idxno=2098943,9079$,9079$아동학대 사망 최대 징역 22년 6개월,[]


In [91]:
dup_list = []
for i in df.title_similar_index:
    dup =  re.findall('\d+',str(i))
    if len(dup) == 0:
        pass
    else:
        dup_list.append(list(map(int, dup))) #숫자로 바꿔줌

In [92]:
flat_dup_list = [item for sublist in dup_list for item in sublist]

In [93]:
flat_dup_list = set(flat_dup_list)
len(flat_dup_list)

1636

In [94]:
df = df.drop(list(flat_dup_list))

In [95]:
df.to_csv('final_df.csv')

#### final_df 로 저장!! 

-----------

In [96]:
df.iloc[1001]

date                                                                   2022.02.13.
title                                          화성시 입양아동 학대사망 사건… 항소심서 양부 "고의성 없었다"
url                    http://www.kyeongin.com/main/view.php?key=20220213010002088
index                                                                        1255$
index_similarity                          1255$화성시 입양아동 학대사망 사건… 항소심서 양부 "고의성 없었다"
title_similar_index                                                             []
Name: 1255, dtype: object

# 비슷한 거들 하나씩만 남기고 삭제하는 방법을 모르겠음..

In [97]:
df.drop(['index','index_similarity','title_similar_index'], axis=1, inplace=True)

In [98]:
df.reset_index(drop=True, inplace=True)

-------

## 3. 처벌 

In [99]:
import re

for i in range(len(df)): 
    punishment = df.loc[i,'title']

    punishment = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', punishment)
    punishment = re.sub('\s{2,}',' ',punishment) 
    punishment =  re.findall('[가-힣]+형\s|사형|무기징역|징역\s*\d*년',punishment)     # 예. 집행유예형 ... 
    punishment = ' '.join(punishment)

    
    df.loc[i,'punish']= punishment

-----------------

## 4. 정규표현식과 명사 (뉴스 핵심 키워드)

In [100]:
from konlpy.tag import Okt
import re

for i in range(len(df)): # df 의 행 수만큼 for 문 돌린다.
    news = df.loc[i,'title'] # news 컬럼만의 각 값을 순서대로 가져온다.
    news =  re.findall('[가-힣]+|[가-힣]+\d+|\d+[가-힣]+',news)    # 가져온 행의 값에서 정규표현식 수행 # 영어 추가..\w
    news = ' '.join(news) # 위에서는 한글과 숫자만 추출된 요소들이 list 안에 있으니 전부를 하나로 묶어줌, 이때 스페이스로 분리
    news = re.sub('^\s+|\s+$|\s{2,}','',news) # 불필요한 공백들을 전부 제거

    okt = Okt()
    morph = okt.pos(news)
    noun_list = []
    for word, tag in morph:
        if (tag=='Noun') & (len(word) > 1):# 형용사 
            noun_list.append(word)
    noun_list= ' '.join(noun_list)
    
    df.loc[i,'nouns']= noun_list

------------

----------

## 5. 가해자 나이대

import re

for i in range(len(df)): 
    news = df.loc[i,'title'] 
    news =  re.search('살해\w*\s(\d+)[대]',news)
    if news:
        news = news.group()
    

    df.loc[i,'age']= news
    
df.head(30)

In [101]:
import re

for i in range(len(df)): # df 의 행 수만큼 for 문 돌린다.
    news = df.loc[i,'title'] # news 컬럼만의 각 값을 순서대로 가져온다.
    news =  re.findall('(\d+)[대]',news)    # 가져온 행의 값에서 정규표현식 수행 # 영어 추가..\w
    
    if len(news)== 0:
        news= None
    
    elif len(news) > 1 : # 두개 나오는 경우가 있는데 보통 가해자는 뒷쪽에 나오기 때문에 두번째 것
        news = news[1]
    else:
        news = news[0]

    df.loc[i,'age']= news
    
df.head(30)

,date,title,url,punish,nouns,age
0,2022.01.31.,'제2의 자두 사건'...고양이 살해범 잡아주세요,https://www.ytn.co.kr/_ln/0115_202201312029383965,,자두 사건 고양이 살해,NaN
1,2022.01.31.,"""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수",http://www.newsis.com/view/?id=NISX20220131_0001743534&cID=10812&pID=10800,,고양이 꼬리 잡고 바닥 살해 경찰 수사 착수,NaN
2,2022.01.31.,"이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”",http://news.kmib.co.kr/article/view.asp?arcid=0016723451&code=61111111&cp=nv,,이재명 고양이 두부 잔혹 살해 사람 수도,NaN
3,2022.01.31.,[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대,https://www.news1.kr/articles/?4570950,,사건 구성 범행 피해자 언니 살해 인면수,30
4,2022.01.30.,"[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?""",https://www.news1.kr/articles/?4570040,,사건 구성 연하 남친 살해 돌싱 창피해,None
5,2022.01.30.,고양이 ‘두부’ 잔혹 살해 범죄에 이재명 “경찰 적극 수사해야”,https://www.hani.co.kr/arti/politics/politics_general/1029321.html,,고양이 두부 잔혹 살해 범죄 이재명 경찰 적극 수사,None
6,2022.01.26.,16세 연하 남친 흉기 살해 30대 여성 징역 22년,https://www.seoul.co.kr/news/newsView.php?id=20220126500100&wlog_tag3=naver,징역 22년,연하 남친 흉기 살해 여성 징역,30
7,2022.01.26.,결혼 1달 만에 아내 폭행·살해한 50대에 징역 30년 구형,http://www.newsis.com/view/?id=NISX20220126_0001738203&cID=10807&pID=10800,징역 30년,결혼 아내 폭행 살해 징역 구형,50
8,2022.01.26.,"“내 번호 왜 지워”… 남친 살해한 여성, 무기징역→징역 22년 감형",https://biz.chosun.com/topics/law_firm/2022/01/26/CICTF4BSGBGEVFNH2M2SFIA7ZI/?utm_source=naver&utm_medium=original&utm_campaign=biz,무기징역 징역 22년,번호 남친 살해 여성 무기징역 징역 감형,None
9,2022.01.30.,창원서 고양이 꼬리 잡고 살해…이재명 “경찰의 적극 수사 요청”,http://www.segye.com/content/html/2022/01/30/20220130506469.html?OutUrl=naver,,창원 고양이 꼬리 잡고 살해 이재명 경찰 적극 수사 요청,None


-------------

## 6. 국내로 제한
1. 제외 대상인 국가 리스트  추출 

In [102]:
#https://ko.wiktionary.org/wiki/%EB%B6%80%EB%A1%9D:%EC%84%B8%EA%B3%84%EC%9D%98_%EB%82%98%EB%9D%BC# Q. 네이버 영화 랭킹 가져와서 첫번째 영화제목을 출력하세요
from bs4 import BeautifulSoup
import re
import pandas
import urllib.request as rq

url = 'https://ko.wiktionary.org/wiki/%EB%B6%80%EB%A1%9D:%EC%84%B8%EA%B3%84%EC%9D%98_%EB%82%98%EB%9D%BC'
html = rq.urlopen(url) # 사이트 열기
bs = BeautifulSoup(html,'html.parser')
countries = bs.find_all('tr') 

country_list = []
for country in countries:
    country=country.find('td') # 첫번째 열에 국가 이름이 있기 때문
    if  country:
        country = country.get_text()
        country = country[:-1] # 이름 마다 끝에 \n 이 있어서 
        country_list.append(country)

country_list.append('아베')
country_list.remove('한국')
country_list.append('중국')

###### country_list

In [103]:
df.head(15)

,date,title,url,punish,nouns,age
0,2022.01.31.,'제2의 자두 사건'...고양이 살해범 잡아주세요,https://www.ytn.co.kr/_ln/0115_202201312029383965,,자두 사건 고양이 살해,NaN
1,2022.01.31.,"""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수",http://www.newsis.com/view/?id=NISX20220131_0001743534&cID=10812&pID=10800,,고양이 꼬리 잡고 바닥 살해 경찰 수사 착수,NaN
2,2022.01.31.,"이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”",http://news.kmib.co.kr/article/view.asp?arcid=0016723451&code=61111111&cp=nv,,이재명 고양이 두부 잔혹 살해 사람 수도,NaN
3,2022.01.31.,[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대,https://www.news1.kr/articles/?4570950,,사건 구성 범행 피해자 언니 살해 인면수,30
4,2022.01.30.,"[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?""",https://www.news1.kr/articles/?4570040,,사건 구성 연하 남친 살해 돌싱 창피해,None
5,2022.01.30.,고양이 ‘두부’ 잔혹 살해 범죄에 이재명 “경찰 적극 수사해야”,https://www.hani.co.kr/arti/politics/politics_general/1029321.html,,고양이 두부 잔혹 살해 범죄 이재명 경찰 적극 수사,None
6,2022.01.26.,16세 연하 남친 흉기 살해 30대 여성 징역 22년,https://www.seoul.co.kr/news/newsView.php?id=20220126500100&wlog_tag3=naver,징역 22년,연하 남친 흉기 살해 여성 징역,30
7,2022.01.26.,결혼 1달 만에 아내 폭행·살해한 50대에 징역 30년 구형,http://www.newsis.com/view/?id=NISX20220126_0001738203&cID=10807&pID=10800,징역 30년,결혼 아내 폭행 살해 징역 구형,50
8,2022.01.26.,"“내 번호 왜 지워”… 남친 살해한 여성, 무기징역→징역 22년 감형",https://biz.chosun.com/topics/law_firm/2022/01/26/CICTF4BSGBGEVFNH2M2SFIA7ZI/?utm_source=naver&utm_medium=original&utm_campaign=biz,무기징역 징역 22년,번호 남친 살해 여성 무기징역 징역 감형,None
9,2022.01.30.,창원서 고양이 꼬리 잡고 살해…이재명 “경찰의 적극 수사 요청”,http://www.segye.com/content/html/2022/01/30/20220130506469.html?OutUrl=naver,,창원 고양이 꼬리 잡고 살해 이재명 경찰 적극 수사 요청,None


2. 국가 이름이 언급된 뉴스 제목들은 제거 

In [104]:
index = 0
delete_rows = []
for i in range(len(df)): 
    nouns = df.loc[i,'nouns'] #제목 내에 명사들
    nouns = nouns.split()
    for noun in nouns: # 각 명사 살펴보기
        if noun in country_list: # 리스트에 있으면 제거 요소로 추가
            print(noun)
            delete_rows.append(index)
        else: 
            pass
        
    index +=1

print(delete_rows)
df.drop(set(delete_rows), inplace=True) # 한 제목에 여러 국가 관련 단어 들어있을 수 잇으니 unique 로 
df

과테말라
중국
카자흐스탄
카자흐스탄
베트남
과테말라
중국
중국
멕시코
베트남
노르웨이
노르웨이
중국
베트남
중국
독일
시리아
시리아
중국
파키스탄
프랑스
피지
프랑스
베트남
카자흐스탄
파키스탄
베트남
파키스탄
스위스
태국
태국
태국
태국
태국
파키스탄
엘살바도르
태국
태국
태국
중국
태국
미국
뉴질랜드
베트남
니카라과
베트남
베트남
베트남
베트남
베트남
베트남
베트남
베트남
베트남
베트남
베트남
중국
멕시코
중국
베네수엘라
멕시코
멕시코
르완다
파키스탄
러시아
르완다
파키스탄
베네수엘라
부르키나파소
엘살바도르
우크라이나
멕시코
우크라이나
러시아
볼리비아
독일
우크라이나
러시아
러시아
독일
러시아
러시아
우크라이나
우크라이나
우크라이나
러시아
미국
멕시코
우크라이나
필리핀
필리핀
이라크
일본
영국
독일
온두라스
온두라스
중국
중국
일본
필리핀
필리핀
필리핀
필리핀
필리핀
필리핀
필리핀
필리핀
미국
필리핀
필리핀
중국
이란
멕시코
프랑스
온두라스
싱가포르
온두라스
필리핀
미국
미국
콜롬비아
브라질
그리스
차드
독일
독일
영국
멕시코
러시아
러시아
중국
독일
르완다
말레이시아
영국
러시아
영국
덴마크
러시아
중국
러시아
페루
일본
베트남
인도
중국
미국
중국
인도
중국
인도
태국
미국
태국
베트남
태국
태국
스위스
독일
독일
노르웨이
프랑스
알제리
파키스탄
파키스탄
노르웨이
그리스
파키스탄
중국
파키스탄
인도
인도
미국
미국
영국
독일
독일
프랑스
프랑스
독일
모로코
프랑스
베냉
[72, 76, 85, 110, 208, 213, 216, 229, 247, 309, 366, 375, 376, 385, 423, 427, 427, 517, 588, 626, 636, 648, 771, 939, 942, 987, 1017, 1022, 1029, 1043, 1070, 1077, 1092, 1109, 1117, 1134, 1139, 1141, 1143, 1169, 1172, 1183, 1228, 1263, 1275, 1341, 1348, 1354, 1360, 1364, 1367

,date,title,url,punish,nouns,age
0,2022.01.31.,'제2의 자두 사건'...고양이 살해범 잡아주세요,https://www.ytn.co.kr/_ln/0115_202201312029383965,,자두 사건 고양이 살해,NaN
1,2022.01.31.,"""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수",http://www.newsis.com/view/?id=NISX20220131_0001743534&cID=10812&pID=10800,,고양이 꼬리 잡고 바닥 살해 경찰 수사 착수,NaN
2,2022.01.31.,"이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”",http://news.kmib.co.kr/article/view.asp?arcid=0016723451&code=61111111&cp=nv,,이재명 고양이 두부 잔혹 살해 사람 수도,NaN
3,2022.01.31.,[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대,https://www.news1.kr/articles/?4570950,,사건 구성 범행 피해자 언니 살해 인면수,30
4,2022.01.30.,"[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?""",https://www.news1.kr/articles/?4570040,,사건 구성 연하 남친 살해 돌싱 창피해,None
...,...,...,...,...,...,...
7440,2021.12.16.,집에 늦게 들어온다고 했다고…아내 흉기로 살해한 40대,https://www.nocutnews.co.kr/news/5674769,,아내 흉기 살해,40
7441,2021.12.01.,"""동물에게도 못할 범행""…검찰, 생후 20개월 학대 살해범에 사형 구형",https://www.nocutnews.co.kr/news/5666447,사형,동물 범행 검찰 생후 개월 학대 살해 사형 구형,None
7442,2021.12.09.,의처증 남편 살해한 여인의 기구한 운명…'미친.사랑.X',http://www.newsis.com/view/?id=NISX20211209_0001681606&cID=10601&pID=10600,,의처증 남편 살해 여인 기구 운명 사랑,None
7443,2021.12.15.,[사건·사고24시] ‘세 모녀 살해’ 김태현 오늘 항소심 첫 재판·“왜 내 아내 또 만나” 내연남 때려 숨지게 한 40대 外,https://www.etoday.co.kr/news/view/2087165,,사건 사고 모녀 살해 김태현 오늘 항소심 재판 아내 연남,40


In [105]:
df.head(50)

,date,title,url,punish,nouns,age
0,2022.01.31.,'제2의 자두 사건'...고양이 살해범 잡아주세요,https://www.ytn.co.kr/_ln/0115_202201312029383965,,자두 사건 고양이 살해,NaN
1,2022.01.31.,"""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수",http://www.newsis.com/view/?id=NISX20220131_0001743534&cID=10812&pID=10800,,고양이 꼬리 잡고 바닥 살해 경찰 수사 착수,NaN
2,2022.01.31.,"이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”",http://news.kmib.co.kr/article/view.asp?arcid=0016723451&code=61111111&cp=nv,,이재명 고양이 두부 잔혹 살해 사람 수도,NaN
3,2022.01.31.,[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대,https://www.news1.kr/articles/?4570950,,사건 구성 범행 피해자 언니 살해 인면수,30
4,2022.01.30.,"[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?""",https://www.news1.kr/articles/?4570040,,사건 구성 연하 남친 살해 돌싱 창피해,None
5,2022.01.30.,고양이 ‘두부’ 잔혹 살해 범죄에 이재명 “경찰 적극 수사해야”,https://www.hani.co.kr/arti/politics/politics_general/1029321.html,,고양이 두부 잔혹 살해 범죄 이재명 경찰 적극 수사,None
6,2022.01.26.,16세 연하 남친 흉기 살해 30대 여성 징역 22년,https://www.seoul.co.kr/news/newsView.php?id=20220126500100&wlog_tag3=naver,징역 22년,연하 남친 흉기 살해 여성 징역,30
7,2022.01.26.,결혼 1달 만에 아내 폭행·살해한 50대에 징역 30년 구형,http://www.newsis.com/view/?id=NISX20220126_0001738203&cID=10807&pID=10800,징역 30년,결혼 아내 폭행 살해 징역 구형,50
8,2022.01.26.,"“내 번호 왜 지워”… 남친 살해한 여성, 무기징역→징역 22년 감형",https://biz.chosun.com/topics/law_firm/2022/01/26/CICTF4BSGBGEVFNH2M2SFIA7ZI/?utm_source=naver&utm_medium=original&utm_campaign=biz,무기징역 징역 22년,번호 남친 살해 여성 무기징역 징역 감형,None
9,2022.01.30.,창원서 고양이 꼬리 잡고 살해…이재명 “경찰의 적극 수사 요청”,http://www.segye.com/content/html/2022/01/30/20220130506469.html?OutUrl=naver,,창원 고양이 꼬리 잡고 살해 이재명 경찰 적극 수사 요청,None


In [106]:
df.reset_index(drop=True, inplace=True)

In [107]:
df.head(20)

,date,title,url,punish,nouns,age
0,2022.01.31.,'제2의 자두 사건'...고양이 살해범 잡아주세요,https://www.ytn.co.kr/_ln/0115_202201312029383965,,자두 사건 고양이 살해,NaN
1,2022.01.31.,"""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수",http://www.newsis.com/view/?id=NISX20220131_0001743534&cID=10812&pID=10800,,고양이 꼬리 잡고 바닥 살해 경찰 수사 착수,NaN
2,2022.01.31.,"이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”",http://news.kmib.co.kr/article/view.asp?arcid=0016723451&code=61111111&cp=nv,,이재명 고양이 두부 잔혹 살해 사람 수도,NaN
3,2022.01.31.,[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대,https://www.news1.kr/articles/?4570950,,사건 구성 범행 피해자 언니 살해 인면수,30
4,2022.01.30.,"[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?""",https://www.news1.kr/articles/?4570040,,사건 구성 연하 남친 살해 돌싱 창피해,None
5,2022.01.30.,고양이 ‘두부’ 잔혹 살해 범죄에 이재명 “경찰 적극 수사해야”,https://www.hani.co.kr/arti/politics/politics_general/1029321.html,,고양이 두부 잔혹 살해 범죄 이재명 경찰 적극 수사,None
6,2022.01.26.,16세 연하 남친 흉기 살해 30대 여성 징역 22년,https://www.seoul.co.kr/news/newsView.php?id=20220126500100&wlog_tag3=naver,징역 22년,연하 남친 흉기 살해 여성 징역,30
7,2022.01.26.,결혼 1달 만에 아내 폭행·살해한 50대에 징역 30년 구형,http://www.newsis.com/view/?id=NISX20220126_0001738203&cID=10807&pID=10800,징역 30년,결혼 아내 폭행 살해 징역 구형,50
8,2022.01.26.,"“내 번호 왜 지워”… 남친 살해한 여성, 무기징역→징역 22년 감형",https://biz.chosun.com/topics/law_firm/2022/01/26/CICTF4BSGBGEVFNH2M2SFIA7ZI/?utm_source=naver&utm_medium=original&utm_campaign=biz,무기징역 징역 22년,번호 남친 살해 여성 무기징역 징역 감형,None
9,2022.01.30.,창원서 고양이 꼬리 잡고 살해…이재명 “경찰의 적극 수사 요청”,http://www.segye.com/content/html/2022/01/30/20220130506469.html?OutUrl=naver,,창원 고양이 꼬리 잡고 살해 이재명 경찰 적극 수사 요청,None


## 뉴스 기사 분석 토대로 만든 살인 범죄 사전 (+유사단어 및 통계에서 얻은 키워드 추가)

In [108]:
crime_dict = pd.read_csv('./범죄사전.csv', encoding='utf-8')
crime_dict.head()

,person,method,reason
0,아내,흉기,생활고
1,남자친구,둔기,잔소리
2,아들,성폭행,빚
3,여친,성추행,장애
4,부부,폭행,복수


## 7. 피해자

In [109]:
for i in range(len(df)): # df 의 행 수만큼 for 문 돌린다.
    victim = df.loc[i,'nouns'] # 컬럼만의 각 값을 순서대로 가져온다.
    victim = list(victim.split())
    
    temp_li = []
    if '살해' in victim:
        index = victim.index('살해') #살해 라는 단어 전 단어들만 추출
        victim = victim[:index]

        if len(victim) > 0:

            for v in victim:
                v = list(v.split())
                for j in v:
                    if j in list(crime_dict['person']):
                        temp_li.append(j)
                    
                            
    
    
    if len(temp_li) > 0:
        print(temp_li)
        temp_li = temp_li[-1] #여러개 나올 경우 살해 바로 전 사람단어로 선택
        df.loc[i,'victim']= temp_li
        
    else:
        
        df.loc[i,'victim']= None
    


['피해자', '언니']
['아내']
['자매']
['이웃']
['아들']
['자매']
['남편']
['자매']
['남편']
['동거']
['여자친구']
['여친']
['고교생']
['자매']
['지인', '여성']
['자매']
['자매']
['남자친구']
['자녀']
['아내']
['아내']
['자매']
['아들']
['친구']
['아내']
['자매']
['여자친구', '언니']
['친구']
['이웃']
['남자친구']
['자매']
['자매']
['이웃']
['친구']
['자매']
['여자친구', '언니']
['자매']
['모친']
['아내']
['친구']
['아내']
['남자친구']
['이웃']
['아내']
['아내']
['친구']
['연인']
['여자친구', '언니']
['여친', '언니']
['지인']
['지인']
['자매']
['지인']
['남자친구']
['자매']
['자매']
['자매']
['여자친구', '언니']
['고교생']
['형제']
['고교생']
['여성']
['친척']
['여친']
['아내']
['남자친구']
['고교생']
['여성']
['남자친구']
['남자친구']
['지인']
['고교생']
['연인']
['아버지', '여자친구']
['아들']
['여성']
['아내']
['여성']
['중학생']
['고교생']
['지인']
['어머니']
['친구']
['아내']
['여성']
['여성']
['어머니']
['고교생']
['여성']
['고교생']
['지인']
['아내']
['언니']
['여친']
['지인']
['아내']
['아들']
['아내']
['고교생']
['아내']
['동거']
['아버지']
['아들']
['자매']
['남편']
['자녀']
['아내']
['아버지']
['아내']
['아버지']
['아내']
['고교생']
['남자친구']
['아버지']
['지인']
['지인']
['여성']
['형제']
['아들']
['자매']
['자녀']
['여성']
['여성']
['아내']
['자매']
['아버지']
['여성']
['동거', '주인']
['

['주인']
['직원']
['아내']
['아버지']
['동생', '오빠']
['아버지']
['부모']
['부부']
['아버지']
['부모', '어머니']
['중학생']
['배우자', '애인']
['아내']
['부모', '노모']
['동생']
['아버지']
['연인']
['애인']
['누나']
['부모', '노모']
['아내']
['아버지']
['연인']
['부부']
['모친']
['아버지']
['부모', '어머니']
['어머니']
['부모', '노모']
['누나']
['자녀', '부부']
['친구']
['친구']
['자녀', '부부']
['모친']
['직원']
['연인']
['누나', '부모']
['직원']
['애인']
['연인']
['누나']
['연인']
['부모']
['아내']
['자녀', '부부']
['친구']
['어머니']
['누나']
['여성']
['누나']
['엄마']
['친구']
['누나']
['누나']
['연인']
['누나']
['중학생', '아들']
['친구']
['여친']
['아내']
['연인']
['누나']
['중학생']
['아버지']
['모친']
['아내']
['친구']
['모친']
['동거']
['어머니']
['동거']
['어머니']
['부부', '자녀']
['지인']
['피해자', '친구']
['아내']
['부모']
['아내']
['직원']
['누나']
['지인']
['모친']
['어머니']
['동거']
['친구']
['지인']
['중학생']
['친구']
['연인']
['부부']
['아버지']
['동생']
['남편', '중학생']
['지인']
['부모', '노모']
['중학생']
['동료']
['친구']
['부부', '자녀']
['지인']
['어머니']
['아들']
['엄마']
['아내']
['중학생']
['여성']
['엄마']
['여친']
['친부']
['지인']
['아들']
['친구']
['연인']
['아내']
['어머니']
['아내']
['아들']
['친구']
['친구']
['부모', '누나']
['아들']
['부모', '엄마']

['아들']
['아들']
['연인']
['남편']
['자녀']
['직원']
['여성']
['자녀']
['아내']
['남편']
['아들']
['아버지']
['아내']
['중학생']
['연인']
['여성']
['동료']
['아내']
['여성']
['아내']
['중학생']
['지인']
['지인']
['여성']
['여성']
['동거']
['자녀']
['여성']
['중학생']
['아버지']
['여성']
['여성', '지인']
['여성']
['동거', '아들']
['아내']
['아들']
['아들']
['지인', '여성']
['남편']
['여성']
['아내']
['남편']
['여성']
['친구']
['연인']
['여친']
['동료']
['지인']
['아버지']
['중학생']
['여성']
['여성', '지인']
['자매']
['여성']
['여자친구']
['아내']
['여성']
['여성']
['동생']
['여친', '자매']
['여자친구']
['아버지']
['아내']
['여성']
['중학생']
['남편']
['동거']
['여자친구']
['여성']
['아내']
['지인']
['어머니']
['노모']
['여성']
['남편']
['아내']
['지인', '여성']
['자녀']
['중학생']
['동료']
['연인']
['동거']
['여성']
['지인']
['아버지']
['중학생']
['자매']
['여친']
['여성']
['남편']
['아내', '아내']
['자매']
['동거']
['여성']
['아버지']
['지인', '여성']
['여성']
['아버지']
['남자친구']
['아내']
['여성']
['여친']
['중학생']
['남편']
['동거']
['장애인']
['동창']
['여성']
['아내']
['남편']
['지인']


## 8. 가해자

In [110]:
for i in range(len(df)): # df 의 행 수만큼 for 문 돌린다.
    victim = df.loc[i,'nouns'] # 컬럼만의 각 값을 순서대로 가져온다.
    victim = list(victim.split())
    
    temp_li = []
    if '살해' in victim:
        index = victim.index('살해') #살해 라는 단어 후 단어들만 추출
        victim = victim[(index+1):]

        if len(victim) > 0:

            for v in victim:
                v = list(v.split())
                for j in v:
                    if j in list(crime_dict['person']):
                        temp_li.append(j)
                    
                            
    
    
    if len(temp_li) > 0:
        print(temp_li)
        temp_li = temp_li[0] #여러개 나올 경우 살해 바로 후 사람단어로 선택
        df.loc[i,'murderer']= temp_li
        
    else:
        
        df.loc[i,'murderer']= None
    

['여성']
['여성']
['직원']
['아버지']
['동생']
['아내']
['형제']
['직원']
['형제', '동생']
['동거']
['아버지']
['동거']
['형제']
['엄마']
['형제']
['어머니']
['형제']
['직원']
['여성']
['직원']
['형제', '동생']
['형제']
['여성']
['동생']
['형제']
['형제']
['동생']
['형제']
['자매']
['형제']
['형제']
['형제']
['형제']
['여성']
['여친']
['동생']
['형제']
['형제']
['형제', '동생']
['동생', '형제']
['형제', '동생']
['형제', '동생']
['형제']
['형제']
['여성']
['여성']
['형제']
['형제']
['형제']
['형제']
['형제']
['여성']
['엄마']
['남편']
['형제']
['형제']
['형제']
['형제']
['형제']
['여성']
['형제']
['형제']
['형제']
['남편']
['친구']
['가해자']
['동거']
['형제']
['남편']
['형제']
['엄마']
['엄마']
['아버지']
['아들']
['형제']
['아들']
['형제']
['여성']
['아들']
['형제']
['형제']
['엄마']
['여성']
['남편']
['아들']
['엄마']
['남편']
['형제', '동생']
['어머니']
['여성']
['남편']
['아들']
['아버지']
['어머니']
['여성']
['아들']
['여성']
['남편']
['남편']
['피해자']
['계부']
['남편']
['아들']
['아들']
['동생']
['남편']
['아들']
['부부']
['오빠']
['어머니']
['남편']
['여성']
['아들']
['아내']
['부부']
['여성']
['가해자']
['남편']
['부부']
['아내']
['아들']
['남편']
['부부']
['아들']
['남편']
['부부']
['부부']
['부부']
['남편']
['남편']
['아들']
['어머니']
['남편']
['아들']
['아들', '

## 9. 피해자와 가해자 둘중 하나가 None일 경우 관계를 파악

In [111]:
import re

for i in range(len(df)): # df 의 행 수만큼 for 문 돌린다.
    victim = df.loc[i,'victim']
    
    
    if df.loc[i,'murderer'] == None: # 비어있을 경우에만 
        if victim == '아내':
            df.loc[i,'murderer'] = '남편'

        if victim == '배우자':
            df.loc[i,'murderer'] = '배우자'

        if victim == '친구':
            df.loc[i,'murderer'] = '친구'

        if victim == '부모':
            df.loc[i,'murderer'] = '자식'

        if victim == '지인':
            df.loc[i,'murderer'] = '지인'
        

In [112]:
import re

for i in range(len(df)): # df 의 행 수만큼 for 문 돌린다.
    murderer = df.loc[i,'murderer']
    
    
    if df.loc[i,'victim'] == None: # 비어있을 경우에만 
        if murderer == '아내':
            df.loc[i,'victim'] = '남편'

        if murderer == '배우자':
            df.loc[i,'victim'] = '배우자'

        if murderer == '친구':
            df.loc[i,'victim'] = '친구'

        if murderer == '부모':
            df.loc[i,'victim'] = '자식'

        if murderer == '지인':
            df.loc[i,'victim'] = '지인'
            
        if murderer == '자식':
            df.loc[i,'victim'] = '부모'
            
        if murderer == '아버지' or murderer == '어머니':
            df.loc[i,'victim'] = '자녀'
        

## 10 . 결과

In [113]:
df

,date,title,url,punish,nouns,age,victim,murderer
0,2022.01.31.,'제2의 자두 사건'...고양이 살해범 잡아주세요,https://www.ytn.co.kr/_ln/0115_202201312029383965,,자두 사건 고양이 살해,NaN,NaN,NaN
1,2022.01.31.,"""고양이 꼬리잡고 바닥에 내리쳐 살해""…경찰 수사 착수",http://www.newsis.com/view/?id=NISX20220131_0001743534&cID=10812&pID=10800,,고양이 꼬리 잡고 바닥 살해 경찰 수사 착수,NaN,NaN,NaN
2,2022.01.31.,"이재명, 고양이 ‘두부’ 잔혹 살해에 “사람 향할 수도”",http://news.kmib.co.kr/article/view.asp?arcid=0016723451&code=61111111&cp=nv,,이재명 고양이 두부 잔혹 살해 사람 수도,NaN,NaN,NaN
3,2022.01.31.,[사건의 재구성]범행 감추려고 피해자 언니까지 살해한 인면수심 30대,https://www.news1.kr/articles/?4570950,,사건 구성 범행 피해자 언니 살해 인면수,30,언니,NaN
4,2022.01.30.,"[사건의 재구성] 16살 연하 남친 살해한 돌싱녀…""내가 창피해?""",https://www.news1.kr/articles/?4570040,,사건 구성 연하 남친 살해 돌싱 창피해,None,None,NaN
...,...,...,...,...,...,...,...,...
7262,2021.12.16.,집에 늦게 들어온다고 했다고…아내 흉기로 살해한 40대,https://www.nocutnews.co.kr/news/5674769,,아내 흉기 살해,40,아내,남편
7263,2021.12.01.,"""동물에게도 못할 범행""…검찰, 생후 20개월 학대 살해범에 사형 구형",https://www.nocutnews.co.kr/news/5666447,사형,동물 범행 검찰 생후 개월 학대 살해 사형 구형,None,None,None
7264,2021.12.09.,의처증 남편 살해한 여인의 기구한 운명…'미친.사랑.X',http://www.newsis.com/view/?id=NISX20211209_0001681606&cID=10601&pID=10600,,의처증 남편 살해 여인 기구 운명 사랑,None,남편,None
7265,2021.12.15.,[사건·사고24시] ‘세 모녀 살해’ 김태현 오늘 항소심 첫 재판·“왜 내 아내 또 만나” 내연남 때려 숨지게 한 40대 外,https://www.etoday.co.kr/news/view/2087165,,사건 사고 모녀 살해 김태현 오늘 항소심 재판 아내 연남,40,남편,아내


In [114]:
preprocessed_df = df[df[['title','url','punish','age','victim','murderer']].notna().all(axis=1)]

In [115]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 573 entries, 7 to 7266
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      573 non-null    object
 1   title     573 non-null    object
 2   url       573 non-null    object
 3   punish    573 non-null    object
 4   nouns     573 non-null    object
 5   age       573 non-null    object
 6   victim    573 non-null    object
 7   murderer  573 non-null    object
dtypes: object(8)
memory usage: 40.3+ KB


In [116]:
preprocessed_df.columns = ['date', 'title', 'url', '처벌','범죄_키워드','가해자_나이대','피해자','가해자']

In [117]:
preprocessed_df.head(50)

,date,title,url,처벌,범죄_키워드,가해자_나이대,피해자,가해자
7,2022.01.26.,결혼 1달 만에 아내 폭행·살해한 50대에 징역 30년 구형,http://www.newsis.com/view/?id=NISX20220126_0001738203&cID=10807&pID=10800,징역 30년,결혼 아내 폭행 살해 징역 구형,50,아내,남편
38,2022.01.29.,"술주정·폭언·욕설 시달려 남편 살해한 60대 아내, '징역 4년'",https://www.news1.kr/articles/?4570429,징역 4년,주정 폭언 욕설 남편 살해 아내 징역,60,남편,아내
78,2022.01.13.,아내 용서에도…흉기로 살해한 40대 '징역 25년' 구형,https://www.nocutnews.co.kr/news/5689758,징역 25년,아내 용서 흉기 살해 징역 구형,40,아내,남편
83,2022.01.26.,이혼 소송 중 장검으로 아내 살해한 40대에 무기징역 구형,https://view.asiae.co.kr/article/2022012615490412989,무기징역,이혼 소송 장검 아내 살해 무기징역 구형,40,아내,남편
88,2022.01.24.,“왜 수익금 안 줘”…동업 친구 살해하려 한 20대 3명 징역형,https://www.seoul.co.kr/news/newsView.php?id=20220124500042&wlog_tag3=naver,,수익금 동업 친구 살해 징역형,20,친구,친구
89,2022.01.26.,"혼인신고 한 달 뒤 아내 살해 혐의 50대… 검찰, 징역 30년 구형",http://moneys.mt.co.kr/news/mwView.php?no=2022012613218073461,징역 30년,혼인신고 아내 살해 혐의 검찰 징역 구형,50,아내,남편
93,2022.01.24.,수익 배분에 불만 품고 동업하는 친구 살해 시도한 20대 징역 4년,http://www.segye.com/content/html/2022/01/24/20220124502333.html?OutUrl=naver,징역 4년,수익 배분 불만 동업 친구 살해 시도 징역,20,친구,친구
115,2022.01.24.,"""왜 수익금 안 줘"" 동업 친구 살해 시도 20대 3명 실형·집유",https://hankookilbo.com/News/Read/A2022012409190003646?did=NA,실형,수익금 동업 친구 살해 시도 실형 집유,20,친구,친구
129,2022.01.26.,"장인 앞에서 아내 살해 40대男 ""평생 속죄""...檢 무기징역 구형",http://news.mt.co.kr/mtview.php?no=2022012616532912423,무기징역,장인 아내 살해 평생 속죄 무기징역 구형,40,아내,남편
135,2022.01.26.,"장인 앞에서 아내 수십차례 찔러 살해한 40대, 검찰 ‘무기징역’ 구형",http://www.segye.com/content/html/2022/01/26/20220126515897.html?OutUrl=naver,무기징역,장인 아내 차례 살해 검찰 무기징역 구형,40,아내,남편


In [118]:
preprocessed_df.reset_index(drop=True, inplace=True)

In [119]:
preprocessed_df.to_csv('preprocessed_df.csv')

## 같은 행에 다른 열에 있는 것들은 범죄 키워드에서 제거 

In [75]:
for i in range(len(preprocessed_df)): # df 의 행 수만큼 for 문 돌린다.
    keywords = preprocessed_df.loc[i,'범죄_키워드'] # 컬럼만의 각 값을 순서대로 가져온다.
    print(keywords)
    keywords = keywords.split()

    for keyword in keywords:
        if keyword == preprocessed_df.loc[i,'피해자']:
            print(keyword)
            keywords.remove(keyword)
        
        if keyword == preprocessed_df.loc[i,'가해자']:
            keywords.remove(keyword)
        
        if keyword == '징역':
            keywords.remove(keyword)
            
            
    preprocessed_df.loc[i,'범죄_키워드']= keywords
                            

결혼 아내 폭행 살해 징역 구형
아내
주정 폭언 욕설 남편 살해 아내 징역
남편
아내 용서 흉기 살해 징역 구형
아내
이혼 소송 장검 아내 살해 무기징역 구형
아내
수익금 동업 친구 살해 징역형
친구


C:\Users\lynn5\AppData\Local\Temp\ipykernel_20820\3620380611.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df.loc[i,'범죄_키워드']= keywords


ValueError: list.remove(x): x not in list

In [76]:
import re

for i in range(len(df)): # df 의 행 수만큼 for 문 돌린다.
    victim = df.loc[i,'nouns'] # 컬럼만의 각 값을 순서대로 가져온다.
    victim = list(victim.split())
    
    temp_li = []
    if '살해' in victim:
        index = victim.index('살해') #살해 라는 단어 후 단어들만 추출
        victim = victim[(index+1):]

        if len(victim) > 0:

            for v in victim:
                v = list(v.split())
                for j in v:
                    if j in list(crime_dict['person']):
                        temp_li.append(j)
                    
                            
    
    
    if len(temp_li) > 0:
        print(temp_li)
        temp_li = temp_li[0] #여러개 나올 경우 살해 바로 후 사람단어로 선택
        df.loc[i,'murderer']= temp_li
        
    else:
        

IndentationError: expected an indented block (2158346184.py, line 29)